## 通过能耗数据反算总阻力RT

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 定义
etaR = 1.0 # 准确计算较复杂，但整体波动小，经验值是1-1.07或者0.98
etaO = 0.60 # 可以算，但需要知道V_A和thruster force T的具体意义，经验值0.55-0.7
etaS = 0.99 # 无计算公式，取经验值，0.99，0.98，0.95-0.96三种情况
etaH = 1.1 # 要计算t, w，1-t/1-w,需要知道力和速度的关系。
SOFC = 200 # g/kwh
eta = etaR*etaO*etaS*etaH

# 求P_effective
df_fuel = pd.read_csv('/raw_data/mrv.csv')
df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date'],format='ISO8601')
df_fuel = df_fuel[df_fuel['daily_voyage_time_mrv'] >= 24]
df_fuel = df_fuel.dropna(axis=0, subset = ['daily_fuel_mrv'])
df_fuel = df_fuel.reset_index(drop=True)
df_fuel = df_fuel.drop(4747)
df_fuel = df_fuel.reset_index(drop=True)
years = df_fuel['input_date'].dt.year
months = df_fuel['input_date'].dt.month
days = df_fuel['input_date'].dt.day
df_fuel['daily_fuel_mrv'] = df_fuel['daily_fuel_mrv']*1000/24/SOFC*1000 # 一天的油耗(吨)乘10的3次方变为kg，转化成每个小时消耗,kg/h，再根据原公式除以(SOFC*10-3)获得功率，单位千瓦
df_fuel['daily_fuel_mrv'] = df_fuel['daily_fuel_mrv']*eta # 计算P effective 单位千瓦
df_fuel = df_fuel.rename(columns = {'daily_fuel_mrv': 'P_e'})
df_fuel.to_csv('/content/Ship-fuel-prediction/processed/mrv_P_effective.csv')

In [ ]:
# 此时还需要知道速度才能求Rtotal,先和AIS数据合并
df_ais = pd.read_csv('/processed/AIS_hour.csv')
df_ais = df_ais.drop("Unnamed: 0", axis=1)
df_ais['postime'] = pd.to_datetime(df_ais['postime'],format='ISO8601')

years = df_ais['postime'].dt.year
months = df_ais['postime'].dt.month
days = df_ais['postime'].dt.day
num = df_ais['num']
for i in range(df_ais.shape[0]):
  temp = df_fuel[(df_fuel['input_date'].dt.year == years[i]) & (df_fuel['input_date'].dt.month == months[i]) & (df_fuel['input_date'].dt.day == days[i]) & (df_fuel['num'] == num[i])]
  pe = temp['P_e']
  if not pe.empty:
    df_ais.loc[i, 'Pe'] = pe.values
  else:
    df_ais.loc[i, 'Pe'] = np.nan

df_ais